In [1]:
import tensorflow as tf
import numpy as np
import PIL.Image as Image
from tqdm import tqdm_notebook
from scipy.ndimage.filters import gaussian_filter

In [2]:
graph = tf.Graph()
with graph.as_default():
    with tf.gfile.FastGFile('tensorflow_inception_graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
input_tensor = graph.get_tensor_by_name('input:0')
layer_names = ['conv2d0', 'conv2d1', 'conv2d2',
                'mixed3a', 'mixed3b',
                'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e',
                'mixed5a', 'mixed5b']
layers = [graph.get_tensor_by_name(name + ":0") for name in layer_names]
session = tf.InteractiveSession(graph=graph)

In [3]:
def fetch_gradient(layer):
        with graph.as_default():
            layer = tf.square(layer)
            tensor_mean = tf.reduce_mean(layer)
            gradient = tf.gradients(tensor_mean, input_tensor)[0]
        return gradient

In [21]:
def get_image_array(image_path):
    image = Image.open(image_path)
    image = np.float32(image)
    return image

def show_image(image_array):
    image_array = np.uint8(np.clip(image_array, 0, 255))
    display(Image.fromarray(image_array))
    
def resize(image_array, factor=None, size=None):
    image_array = np.uint8(np.clip(image_array, 0, 255))
    if factor is not None:
        size = np.array(image_array.shape[0:2])
        size = size * factor
        size = size.astype(int)
    else:
        size = size[0:2]
    size = tuple(reversed(size))
    image = Image.fromarray(image_array)
    image = image.resize(size, Image.LANCZOS)
    return np.float32(image)

def save_image(image, image_name):
    image = np.clip(image, 0.0, 255.0)
    image = image.astype(np.uint8)
    with open(image_name, 'wb') as f:
        Image.fromarray(image).save(f, 'jpeg')

In [5]:
def dream(layer, image, iterations, modification_rate):
    output = image.copy()
    gradient = fetch_gradient(layer)
    image = np.expand_dims(image, axis=0)
    feed_dict = {input_tensor : image}
    for _ in range(iterations):
        computed_gradient = session.run(gradient, feed_dict=feed_dict)
        blur = 0.8
        computed_gradient = gaussian_filter(computed_gradient, sigma=blur)
        computed_gradient = np.squeeze(computed_gradient)
        output += computed_gradient * modification_rate
    return output

In [6]:
def dream_within_a_dream(layer, image, dream_depth, iterations, modification_rate, factor, mix_factor, blur):
    if dream_depth > 0:
        blurred_image = gaussian_filter(image, sigma=(blur, blur, 0))
        downscaled_image = resize(blurred_image, factor)
        result = dream_within_a_dream(layer,
                                      downscaled_image,
                                      dream_depth-1,
                                      iterations,
                                      modification_rate,
                                      factor,
                                      mix_factor,
                                      blur)
        upscaled_image = resize(result, size=image.shape)
        image = mix_factor*image + (1-mix_factor)*upscaled_image
    result = dream(layer, image, iterations, modification_rate)
    return result
    

In [ ]:
image_name = 'thanos.jpg'
image = get_image_array(image_name)
prev = []
for x in range(65):
    random = np.random.randint(3, 12)
    if random in prev:
        continue
    if x % 4 == 0:
        prev = []
    print('Maximizing layer: ', random)
    image = dream_within_a_dream(layer=layers[random],
                         image=image,
                         dream_depth=5,
                         iterations=10,
                         modification_rate=20,
                         factor=0.7,
                         mix_factor=0.2,
                         blur=0.01)
    save_image(image,f'results/dream_within_a_dream/{image_name}-{x}')
    prev.append(random)

Maximizing layer:  8
Maximizing layer:  5
Maximizing layer:  9
Maximizing layer:  11
